In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import meep as mp
import gmeep as gm

In [ ]:
import pathlib
import tempfile
from typing import Dict, Union
import pydantic

import meep as mp
from meep import mpb

from gmeep.config import disable_print, enable_print
from gmeep.types import Mode

mpb.Verbosity(0)

tmp = pathlib.Path(tempfile.TemporaryDirectory().name).parent / "meep"
tmp.mkdir(exist_ok=True)


@pydantic.validate_arguments
def get_mode_solver_rib(
    wg_width: float = 0.45,
    wg_thickness: float = 0.22,
    slab_thickness: int = 0.0,
    ncore: float = 3.47,
    nclad: float = 1.44,
    sx: float = 2.0,
    sy: float = 2.0,
    res: int = 32,
    nmodes: int = 4,
):
    """Returns a mode_solver simulation.

    Args:
        wg_width: wg_width (um)
        wg_thickness: wg height (um)
        slab_thickness: thickness for the waveguide slab
        ncore: core material refractive index
        nclad: clad material refractive index
        sx: simulation region width (um)
        sy: simulation region height (um)
        res: resolution (pixels/um)
        nmodes: number of modes
    """
    material_core = mp.Medium(index=ncore)
    material_clad = mp.Medium(index=nclad)

    # Define the computational cell.  We'll make x the propagation direction.
    # the other cell sizes should be big enough so that the boundaries are
    # far away from the mode field.
    geometry_lattice = mp.Lattice(size=mp.Vector3(0, sx, sy))

    # define the 2d blocks for the strip and substrate
    geometry = [
        mp.Block(
            size=mp.Vector3(mp.inf, mp.inf, mp.inf),
            material=material_clad,
        ),
        # uncomment this for air cladded waveguides
        # mp.Block(
        #     size=mp.Vector3(mp.inf, mp.inf, 0.5 * (sy - wg_thickness)),
        #     center=mp.Vector3(z=0.25 * (sy + wg_thickness)),
        #     material=material_clad,
        # ),
        mp.Block(
            size=mp.Vector3(mp.inf, mp.inf, slab_thickness),
            material=material_core,
            center=mp.Vector3(z=-0.5 * slab_thickness),
        ),
        mp.Block(
            size=mp.Vector3(mp.inf, wg_width, wg_thickness),
            material=material_core,
            center=mp.Vector3(z=-0.5 * wg_thickness),
        ),
    ]

    # The k (i.e. beta, i.e. propagation constant) points to look at, in
    # units of 2*pi/um.  We'll look at num_k points from k_min to k_max.
    num_k = 9
    k_min = 0.1
    k_max = 3.0
    k_points = mp.interpolate(num_k, [mp.Vector3(k_min), mp.Vector3(k_max)])

    # Increase this to see more modes.  (The guided ones are the ones below the
    # light line, i.e. those with frequencies < kmag / 1.45, where kmag
    # is the corresponding column in the output if you grep for "freqs:".)
    # use this prefix for output files

    filename_prefix = tmp / f"rib_{wg_width}_{wg_thickness}_{slab_thickness}"

    mode_solver = mpb.ModeSolver(
        geometry_lattice=geometry_lattice,
        geometry=geometry,
        k_points=k_points,
        resolution=res,
        num_bands=nmodes,
        filename_prefix=str(filename_prefix),
    )
    return mode_solver


@pydantic.validate_arguments
def find_modes(
    get_mode_solver=get_mode_solver_rib,
    tol: float = 1e-6,
    wavelength: float = 1.55,
    mode_number: int = 1,
    parity=mp.NO_PARITY,
) -> Dict[str, Union[mpb.ModeSolver, float]]:
    """Returns effective index and group index for a mode.

    Args:
        mode_solver_function: function that returns mpb.ModeSolver
        wg_width: wg_width (um)
        wg_thickness: wg height (um)
        ncore: core material refractive index
        nclad: clad material refractive index
        sx: supercell width (um)
        sy: supercell height (um)
        res: (pixels/um)
        wavelength: wavelength
        mode_number: mode order
        paririty: mp.ODD_Y mp.EVEN_X for TE, mp.EVEN_Y for TM. Reduces spurious modes.

    Returns: Dict
        mode_solver
        neff
        ng


    compute mode_number lowest frequencies as a function of k. Also display
    "parities", i.e. whether the mode is symmetric or anti_symmetric
    through the y=0 and z=0 planes.
    mode_solver.run(mpb.display_yparities, mpb.display_zparities)

    Above, we outputed the dispersion relation: frequency (omega) as a
    function of wavevector kx (beta).  Alternatively, you can compute
    beta for a given omega -- for example, you might want to find the
    modes and wavevectors at a fixed wavelength of 1.55 microns.  You
    can do that using the find_k function:
    """
    mode_solver = get_mode_solver()
    omega = 1 / wavelength

    # Output the x component of the Poynting vector for mode_number bands at omega
    disable_print()
    k = mode_solver.find_k(
        parity,
        omega,
        mode_number,
        mode_number,
        mp.Vector3(1),
        tol,
        omega * 2.02,
        omega * 0.01,
        omega * 10,
        mpb.output_poynting_x,
        mpb.display_yparities,
        mpb.display_group_velocities,
    )
    enable_print()
    vg = mode_solver.compute_group_velocities()
    k = k[0]
    vg = vg[0][0]
    neff = wavelength * k
    ng = 1 / vg
    return Mode(neff=neff, ng=ng, solver=mode_solver)



In [ ]:
get_mode_solver=get_mode_solver_rib
tol: float = 1e-6
wavelength: float = 1.55
mode_number: int = 1
parity=mp.NO_PARITY

mode_solver = get_mode_solver()
omega = 1 / wavelength

# Output the x component of the Poynting vector for mode_number bands at omega
disable_print()
k = mode_solver.find_k(
    parity,
    omega,
    mode_number,
    mode_number+2,
    mp.Vector3(1),
    tol,
    omega * 2.02,
    omega * 0.01,
    omega * 10,
    mpb.output_poynting_x,
    mpb.display_yparities,
    mpb.display_group_velocities,
)
enable_print()
vg = mode_solver.compute_group_velocities()
k0 = k[0]
vg = vg[0][0]
neff = wavelength * k0
ng = 1 / vg


In [ ]:
np.array(k)*wavelength

# MPB modes

MPB has periodic boundary conditions.


In [ ]:
# plot TE mode (plot_mode_order=1)
m = gm.find_modes(wg_width=0.4, ncore=3.47, nclad=1.44, wg_thickness=.22, res=32, sx=6, sy=6, mode_number=1, parity=mp.NO_PARITY)
m

In [ ]:
gm.plot_modes(r.solver)

In [ ]:
# plot TM mode (plot_mode_order=2)
r = gm.find_modes(wg_width=0.5, ncore=3.47, nclad=1.44, wg_thickness=.22, res=32, sx=6, sy=6, mode_number=2, parity=mp.NO_PARITY)
r

In [ ]:
gm.plot_modes(r['mode_solver'])

In [ ]:
# plot TM mode (plot_mode_order=1)
import gmeep as gm
import meep as mp

r = gm.find_modes(wg_width=0.5, ncore=3.47, nclad=1.44, wg_thickness=.22, res=32, sx=6, sy=6, parity=mp.EVEN_Y, mode_number=1)
r

In [ ]:
gm.plot_modes(r['mode_solver'])

In [ ]:
# TM mode has some issue if no parity is applied
#neff, ng = gm.find_modes(wg_width=0.5, ncore=3.47, nclad=1.44, wg_thickness=.22, plot=True, res=32, sx=6, sy=6, mode_number=2)
#neff, ng

In [ ]:
import gmeep as gm
import meep as mp
r = gm.find_modes(wg_width=0.8, ncore=3.47, nclad=1.44, wg_thickness=.22, res=32, sx=6, sy=6, mode_number=1, parity=mp.EVEN_Y)
r

In [ ]:
wg_widths = np.arange(200, 2000, 100)*1e-3
n_te0 = [gm.find_modes(wg_width=wg_width, ncore=3.55, wg_thickness=.22) for wg_width in wg_widths]

In [ ]:
neffs_te0 = [ni['neff'] for ni in n_te0]
ngs_te0 = [ni['ng'] for ni in n_te0]

plt.plot(wg_widths, neffs_te0, '.-')
plt.xlabel('wg_width')
plt.ylabel('neff')
plt.title('Silicon strip waveguide (0nm slab)')

In [ ]:
plt.plot(wg_widths, ngs_te0)
plt.xlabel('wg_width')
plt.ylabel('ng')

In [ ]:
wg_widths = np.arange(200, 2000, 100)*1e-3
n_tm0 = [gm.find_modes(wg_width=wg_width, ncore=3.55, wg_thickness=.22, parity=mp.EVEN_Y) for wg_width in wg_widths]

In [ ]:
neffs_tm0 = [ni['neff'] for ni in n_tm0]
ngs_tm0 = [ni['ng'] for ni in n_tm0]

plt.plot(wg_widths, neffs_te0, '.-', label='TE0')
plt.plot(wg_widths, neffs_tm0, '.-', label='TM0')
plt.xlabel('wg_width')
plt.ylabel('neff')
plt.title('Silicon strip waveguide (0nm slab)')
plt.legend()

# Rib waveguides

In [ ]:
# plot TE mode (plot_mode_order=1)
r = gm.find_modes(wg_width=0.5, ncore=3.55, wg_thickness=.22, slab_thickness=90e-3, res=128)
gm.plot_modes(r['mode_solver'])
r

In [ ]:
neff, ng  = r['neff'], r['ng']
print(neff, ng)

In [ ]:
wg_widths = np.arange(200, 2000, 100)*1e-3
n_te0 = [gm.find_modes(wg_width=wg_width, ncore=3.55, wg_thickness=.22, slab_thickness=90e-3) for wg_width in wg_widths]
neffs_te0 = [ni['neff'] for ni in n_te0]
ngs_te0 = [ni['ng'] for ni in n_te0]

In [ ]:
plt.title('Silicon rib waveguide (90nm slab)')
plt.plot(wg_widths, neffs_te0, '.-')
plt.xlabel('wg_width')
plt.ylabel('neff')

## Dispersion

In [ ]:
from gmeep.find_neff_ng_dw_dh import plot_neff_ng_dw_dh
import pandas as pd
import pathlib
from scipy.interpolate import interp2d
import numpy as np
import gmeep as gm

In [ ]:
plot_neff_ng_dw_dh(with_dispersion=True)

In [ ]:
plot_neff_ng_dw_dh(with_dispersion=False)

## Convergence tests

Before launching a set of simulations you need to make sure you have the correct simulation settings:

- res: resolution
- sx: Size of the simulation region in the x-direction (default=4.0)
- sy: Size of the simulation region in the y-direction (default=4.0)


In [ ]:
resolutions = np.linspace(10, 100, 50)
neffs = []
ngs= []

for res in resolutions:
    r = gm.find_modes(wg_width=0.5, ncore=3.5, nclad=1.44, wg_thickness=.22, res=res)
    ngs.append(r['ng'])
    neffs.append(r['neff'])

In [ ]:
plt.plot(resolutions, ngs, 'o-')
plt.ylabel('ng')
plt.xlabel('resolution (pixels/um)')

In [ ]:
plt.plot(resolutions, neffs, 'o-')
plt.ylabel('neff')
plt.xlabel('resolution (pixels/um)')

In [ ]:
sxs = np.linspace(4, 6, 6)
neffs = []
ngs= []

for sx in sxs:
    r = gm.find_modes(
        wg_width=0.5, ncore=3.5, nclad=1.44, wg_thickness=.22, res=20, sx=sx
    )
    ngs.append(r['ng'])
    neffs.append(r['neff'])

In [ ]:
plt.plot(sxs, neffs, 'o-')
plt.ylabel('neff')
plt.xlabel('simulation size in x(um)')

In [ ]:
sys = np.linspace(2, 6, 6)
neffs = []
ngs= []

for sy in sys:
    r = gm.find_modes(
        wg_width=0.5, ncore=3.5, nclad=1.44, wg_thickness=.22, res=20, sy=sy
    )
    ngs.append(r['ng'])
    neffs.append(r['neff'])

In [ ]:
plt.plot(sxs, neffs, 'o-')
plt.ylabel('neff')
plt.xlabel('simulation size in y (um)')